In [1]:
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact
from scipy import stats
import copy

from cvrp_simulation.scenario_generator import SampleDynamicBenchmark
from cvrp_simulation.simulator import CVRPSimulation


In [21]:
depot_position_rv = stats.uniform(loc=0, scale=1)
vehicle_position_rv = stats.uniform(loc=0, scale=1)
customer_positions_rv = stats.uniform(loc=0, scale=1)
customer_demands_rv = stats.uniform(loc=0, scale=10)
customer_times_rv = stats.randint(low=0, high=5)
vehicle_velocity = 10
vrp_size = 20
initial_vehicle_capacity = vrp_size*8
benchmark_generator = SampleDynamicBenchmark(depot_position_rv=depot_position_rv, vehicle_position_rv=vehicle_position_rv, 
                                            vehicle_capacity=initial_vehicle_capacity, vehicle_velocity=vehicle_velocity,
                                            customer_positions_rv=customer_positions_rv, customer_demands_rv=customer_demands_rv,
                                            customer_times_rv=customer_times_rv,
                                            vrp_size=vrp_size)
seed = 5
sim = CVRPSimulation(max_customers=vrp_size, problem_generator=benchmark_generator)
sim.seed(seed)

C:\Users\ros1tv\Anaconda3\envs\waferfab_test\lib\site-packages\gym-0.15.7-py3.7.egg\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [22]:
obs = sim.reset()
# in this case each time we choose action 0 since the available actions change each time
# the number of available customers changes
tot_reward = 0
done = False
all_states = [copy.deepcopy(sim)]
all_observations = [copy.deepcopy(obs)]
timeline = [0]
while not done:
    num_available_actions = sim.get_available_customers().size
    if num_available_actions>0:
        obs, reward, done, _ = sim.step(np.random.randint(0, num_available_actions))
    else:
        obs, reward, done, _ = sim.step(1)
    all_states.append(copy.deepcopy(sim))
    all_observations.append(copy.deepcopy(obs))
    timeline.append(int(np.ceil(sim.current_time)))
    tot_reward += reward
print(f"finished random run, total reward {tot_reward}")


finished random run, total reward -11.000349552058038


In [23]:
@interact
# for range (min, max, step) , this gives a slider
# for drop down give a list of the options
def plot_vehicle_routes(t=(0, len(timeline), 1)):
    plt.close()
    cur_sim = all_states[t]
    observation_space = all_observations[t]
    num_customers = cur_sim.current_state.customer_positions.shape[0]
    plt.scatter([], [], label='opened, demand<capacity',c='b')
    plt.scatter([], [], label='visited', c='g')
    plt.scatter([], [], label='opened, demand>capacity', c='m')
    plt.scatter([], [], label='closed', c='r')
    for i_c in range(num_customers):
        customer_position = cur_sim.current_state.customer_positions[i_c, :]
        # opened customer
        plt.text(customer_position[0], customer_position[1],str(i_c))
        if cur_sim.current_state.customer_visited[i_c]:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='g')
        elif cur_sim.current_state.customer_times[i_c]<= timeline[t]:
            if cur_sim.current_state.customer_demands[i_c]<=sim.current_state.current_vehicle_capacity:
                plt.scatter(customer_position[0], customer_position[1], marker='o', c='b')
            else:
                plt.scatter(customer_position[0], customer_position[1], marker='o', c='m')
        elif cur_sim.current_state.customer_times[i_c]> timeline[t]:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='r')
        else:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='k')
    n_e = 0
    for i_c_o, obs_customer_position in enumerate(observation_space['customer_positions']):
        if obs_customer_position.all()!=0:
            plt.scatter(obs_customer_position[0], obs_customer_position[1], marker='.', c='w')
            plt.text(obs_customer_position[0], obs_customer_position[1], str(i_c_o))
            n_e += 1
    plt.grid()
    plt.legend(loc='best')
    plt.title(f"t={timeline[t]} - customer locations")
    plt.show()
    print(n_e)

interactive(children=(IntSlider(value=10, description='t', max=21), Output()), _dom_classes=('widget-interact'…

In [ ]:
@interact
# for range (min, max, step) , this gives a slider
# for drop down give a list of the options
def plot_current_obs(t=list(range(10))):
    plt.close()
    plt.scatter([2], [3+t])
    plt.xlim([0, 10])
    plt.show()